In [1]:
import os
import sys
gpu_num = 0 # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print("Current directory:", os.getcwd())
# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    sys.path.append("/home/wzs/Project/sionna-main/")
    import sionna as sn

# Import TensorFlow and NumPy
import tensorflow as tf
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')
import numpy as np

# For plotting
%matplotlib inline
import matplotlib.pyplot as plt

# For saving complex Python data structures efficiently
import pickle

# For the implementation of the neural receiver
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Layer

Current directory: /home/wzs/Project/sionna-main/comcloak/training


In [2]:
from sionna.mimo import StreamManagement

from sionna.ofdm import ResourceGrid, ResourceGridMapper, LSChannelEstimator, LMMSEEqualizer
from sionna.ofdm import OFDMModulator, OFDMDemodulator, ZFPrecoder, RemoveNulledSubcarriers

from sionna.channel.tr38901 import AntennaArray, CDL, TDL, Antenna
from sionna.channel import subcarrier_frequencies, cir_to_ofdm_channel, cir_to_time_channel, time_lag_discrete_time_channel
from sionna.channel import ApplyOFDMChannel, ApplyTimeChannel, OFDMChannel, TimeChannel

from sionna.fec.ldpc.encoding import LDPC5GEncoder
from sionna.fec.ldpc.decoding import LDPC5GDecoder

from sionna.mapping import Mapper, Demapper

from sionna.utils import BinarySource, ebnodb2no, sim_ber
from sionna.utils.metrics import compute_ber

In [3]:
# Define the number of UT and BS antennas.
# For the CDL model, that will be used in this notebook, only
# a single UT and BS are supported.
num_ut = 1
num_bs = 1
num_ut_ant = 8
num_bs_ant = 16

num_streams_per_tx = 2

# Create an RX-TX association matrix
# rx_tx_association[i,j]=1 means that receiver i gets at least one stream
# from transmitter j. Depending on the transmission direction (uplink or downlink),
# the role of UT and BS can change. However, as we have only a single
# transmitter and receiver, this does not matter:
rx_tx_association = np.array([[1]])

# Instantiate a StreamManagement object
# This determines which data streams are determined for which receiver.
# In this simple setup, this is fairly easy. However, it can get more involved
# for simulations with many transmitters and receivers.
sm = StreamManagement(rx_tx_association, num_streams_per_tx)
rg = ResourceGrid(num_ofdm_symbols=14,
                  fft_size=64,
                  subcarrier_spacing=15e3,
                  num_tx=1,
                  num_streams_per_tx=num_streams_per_tx,
                  cyclic_prefix_length=16,
                  num_guard_carriers=[5,6],
                  dc_null=True,
                  pilot_pattern="kronecker",
                  pilot_ofdm_symbol_indices=[2,11])

In [4]:
carrier_frequency = 2.6e9 # Carrier frequency in Hz.
                          # This is needed here to define the antenna element spacing.

ut_array = AntennaArray(num_rows=1,
                        num_cols=int(num_ut_ant/2),
                        polarization="dual",
                        polarization_type="cross",
                        antenna_pattern="38.901",
                        carrier_frequency=carrier_frequency)
# ut_array.show()

bs_array = AntennaArray(num_rows=1,
                        num_cols=int(num_bs_ant/2),
                        polarization="dual",
                        polarization_type="cross",
                        antenna_pattern="38.901",
                        carrier_frequency=carrier_frequency)
# bs_array.show()

delay_spread = 300e-9 # Nominal delay spread in [s]. Please see the CDL documentation
                      # about how to choose this value. 

direction = "downlink"  # The `direction` determines if the UT or BS is transmitting.
                      # In the `uplink`, the UT is transmitting.
cdl_model = "B"       # Suitable values are ["A", "B", "C", "D", "E"]
tdl_model = "B"       # Suitable values are ["A", "B", "C", "D", "E"]
speed = 1            # UT speed [m/s]. BSs are always assumed to be fixed.
                      # The direction of travel will chosen randomly within the x-y plane.

# Configure a channel impulse reponse (CIR) generator for the CDL model.
# cdl() will generate CIRs that can be converted to discrete time or discrete frequency.
cdl = CDL(cdl_model, delay_spread, carrier_frequency, ut_array, bs_array, direction, min_speed=speed)
tdl = TDL(tdl_model, delay_spread, carrier_frequency,
                  num_rx_ant=num_ut_ant, num_tx_ant=num_bs_ant, min_speed=speed)
a1, tau1 = cdl(batch_size=8, num_time_steps=rg.num_ofdm_symbols, sampling_frequency=1/rg.ofdm_symbol_duration)
a2, tau2 = tdl(batch_size=8, num_time_steps=rg.num_ofdm_symbols, sampling_frequency=1/rg.ofdm_symbol_duration)
print("Shape of the cdl path gains: ", a1.shape)
print("Shape of the cdl delays:", tau1.shape)
print("Shape of the tdl path gains: ", a2.shape)
print("Shape of the tdl delays:", tau2.shape)
frequencies = subcarrier_frequencies(rg.fft_size, rg.subcarrier_spacing)
h_freq1 = cir_to_ofdm_channel(frequencies, a1, tau1, normalize=True)
h_freq2 = cir_to_ofdm_channel(frequencies, a1, tau1, normalize=True)
# Function that will apply the channel frequency response to an input signal
channel_freq = ApplyOFDMChannel(add_awgn=True)


Shape of the cdl path gains:  (8, 1, 8, 1, 16, 23, 14)
Shape of the cdl delays: (8, 1, 1, 23)
Shape of the tdl path gains:  (8, 1, 8, 1, 16, 23, 14)
Shape of the tdl delays: (8, 1, 1, 23)


In [5]:
NUM_BITS_PER_SYMBOL = 2
BATCH_SIZE = 128 # How many examples are processed by Sionna in parallel
EBN0_DB_MIN = -15.0
EBN0_DB_MAX = 25.0


###############################
# Baseline
###############################

class CDLBaseline(Model): # Inherits from Keras Model

    def __init__(self, num_bits_per_symbol=NUM_BITS_PER_SYMBOL, coderate=0.5):

        super().__init__() # Must call the Keras model initializer

        self.n = int(rg.num_data_symbols*num_bits_per_symbol) # Number of coded bits
        self.k = int(self.n*coderate) # Number of information bits

        # The binary source will create batches of information bits
        self.binary_source = BinarySource()

        # The encoder maps information bits to coded bits
        self.encoder = LDPC5GEncoder(self.k, self.n)

        # The mapper maps blocks of information bits to constellation symbols
        self.mapper = Mapper("qam", num_bits_per_symbol)

        # The resource grid mapper maps symbols onto an OFDM resource grid
        self.rg_mapper = ResourceGridMapper(rg)

        # The zero forcing precoder precodes the transmit stream towards the intended antennas
        self.zf_precoder = ZFPrecoder(rg, sm, return_effective_channel=True)

        # This function removes nulled subcarriers from any tensor having the shape of a resource grid
        self.remove_nulled_scs = RemoveNulledSubcarriers(rg)

        # The LS channel estimator will provide channel estimates and error variances
        self.ls_est = LSChannelEstimator(rg, interpolation_type="nn")

        # The LMMSE equalizer will provide soft symbols together with noise variance estimates
        self.lmmse_equ = LMMSEEqualizer(rg, sm)

        # The demapper produces LLR for all coded bits
        self.demapper = Demapper("app", "qam", num_bits_per_symbol)

        # The decoder provides hard-decisions on the information bits
        self.decoder = LDPC5GDecoder(self.encoder, hard_out=True)
    @tf.function # Enable graph execution to speed things up
    def __call__(self, batch_size, ebno_db):

        # no channel coding used; we set coderate=1.0
        no = sn.utils.ebnodb2no(ebno_db,
                                num_bits_per_symbol=NUM_BITS_PER_SYMBOL,
                                coderate=1.0)
        b = self.binary_source([batch_size, 1, rg.num_streams_per_tx, self.encoder.k])
        c = self.encoder(b)
        x = self.mapper(c)
        x_rg = self.rg_mapper(x)
        

        # As explained above, we generate random batches of CIR, transform them
        # in the frequency domain and apply them to the resource grid in the
        # frequency domain.
        cir = cdl(batch_size, rg.num_ofdm_symbols, 1/rg.ofdm_symbol_duration)
        h_freq = cir_to_ofdm_channel(frequencies, *cir, normalize=True)
        x_rg, g = self.zf_precoder([x_rg, h_freq])
        y = channel_freq([x_rg, h_freq, no])
        h_hat, err_var = self.ls_est([y, no])

        x_hat, no_eff = self.lmmse_equ([y, h_hat, err_var, no])
        llr = self.demapper([x_hat, no_eff])
        b_hat = self.decoder(llr)
        return b, b_hat

class TDLBaseline(Model): # Inherits from Keras Model

    def __init__(self, num_bits_per_symbol=NUM_BITS_PER_SYMBOL, coderate=0.5):

        super().__init__() # Must call the Keras model initializer

        self.n = int(rg.num_data_symbols*num_bits_per_symbol) # Number of coded bits
        self.k = int(self.n*coderate) # Number of information bits

        # The binary source will create batches of information bits
        self.binary_source = BinarySource()

        # The encoder maps information bits to coded bits
        self.encoder = LDPC5GEncoder(self.k, self.n)

        # The mapper maps blocks of information bits to constellation symbols
        self.mapper = Mapper("qam", num_bits_per_symbol)

        # The resource grid mapper maps symbols onto an OFDM resource grid
        self.rg_mapper = ResourceGridMapper(rg)

        # The zero forcing precoder precodes the transmit stream towards the intended antennas
        self.zf_precoder = ZFPrecoder(rg, sm, return_effective_channel=True)

        # This function removes nulled subcarriers from any tensor having the shape of a resource grid
        self.remove_nulled_scs = RemoveNulledSubcarriers(rg)

        # The LS channel estimator will provide channel estimates and error variances
        self.ls_est = LSChannelEstimator(rg, interpolation_type="nn")

        # The LMMSE equalizer will provide soft symbols together with noise variance estimates
        self.lmmse_equ = LMMSEEqualizer(rg, sm)

        # The demapper produces LLR for all coded bits
        self.demapper = Demapper("app", "qam", num_bits_per_symbol)

        # The decoder provides hard-decisions on the information bits
        self.decoder = LDPC5GDecoder(self.encoder, hard_out=True)
    @tf.function # Enable graph execution to speed things up
    def __call__(self, batch_size, ebno_db):

        # no channel coding used; we set coderate=1.0
        no = sn.utils.ebnodb2no(ebno_db,
                                num_bits_per_symbol=NUM_BITS_PER_SYMBOL,
                                coderate=1.0)
        b = self.binary_source([batch_size, 1, rg.num_streams_per_tx, self.encoder.k])
        c = self.encoder(b)
        x = self.mapper(c)
        x_rg = self.rg_mapper(x)
        

        # As explained above, we generate random batches of CIR, transform them
        # in the frequency domain and apply them to the resource grid in the
        # frequency domain.
        cir = tdl(batch_size, rg.num_ofdm_symbols, 1/rg.ofdm_symbol_duration)
        h_freq = cir_to_ofdm_channel(frequencies, *cir, normalize=True)
        x_rg, g = self.zf_precoder([x_rg, h_freq])
        y = channel_freq([x_rg, h_freq, no])
        h_hat, err_var = self.ls_est([y, no])

        x_hat, no_eff = self.lmmse_equ([y, h_hat, err_var, no])
        llr = self.demapper([x_hat, no_eff])
        b_hat = self.decoder(llr)
        return b, b_hat



In [6]:
###############################
# Benchmarking
###############################

baselineQPSK1 = CDLBaseline(2)
baseline16QAM1 = CDLBaseline(4)
baseline64QAM1 = CDLBaseline(6)
ber_plots = sn.utils.PlotBER("cdl baseline Simulation")

ber_plots.simulate(baselineQPSK1,
                  ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 41),
                  batch_size=BATCH_SIZE,
                  num_target_block_errors=100, # simulate until 100 block errors occured
                  legend="Baseline",
                  soft_estimates=True,
                  max_mc_iter=30, # run 100 Monte-Carlo simulations (each with batch_size samples)
                  show_fig=False);
ber_plots.simulate(baseline16QAM1,
                  ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 41),
                  batch_size=BATCH_SIZE,
                  num_target_block_errors=100, # simulate until 100 block errors occured
                  legend="Baseline",
                  soft_estimates=True,
                  max_mc_iter=30, # run 100 Monte-Carlo simulations (each with batch_size samples)
                  show_fig=False);
ber_plots.simulate(baseline64QAM1,
                  ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 41),
                  batch_size=BATCH_SIZE,
                  num_target_block_errors=100, # simulate until 100 block errors occured
                  legend="Baseline",
                  soft_estimates=True,
                  max_mc_iter=30, # run 100 Monte-Carlo simulations (each with batch_size samples)
                  show_fig=False);

EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
    -15.0 | 4.7912e-01 | 1.0000e+00 |       76537 |      159744 |          256 |         256 |         8.2 |reached target block errors
    -14.0 | 4.7376e-01 | 1.0000e+00 |       75681 |      159744 |          256 |         256 |         0.2 |reached target block errors
    -13.0 | 4.6688e-01 | 1.0000e+00 |       74582 |      159744 |          256 |         256 |         0.4 |reached target block errors
    -12.0 | 4.6024e-01 | 1.0000e+00 |       73520 |      159744 |          256 |         256 |         0.4 |reached target block errors
    -11.0 | 4.4524e-01 | 1.0000e+00 |       71125 |      159744 |          256 |         256 |         0.2 |reached target block errors
    -10.0 | 4.3998e-01 | 1.0000e+00 |       70284 |      159744 |

In [7]:
###############################
# Benchmarking
###############################

baselineQPSK2 = TDLBaseline(2)
baseline16QAM2 = TDLBaseline(4)
baseline64QAM2 = TDLBaseline(6)
ber_plots = sn.utils.PlotBER("tdl baseline Simulation")

ber_plots.simulate(baselineQPSK2,
                  ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 41),
                  batch_size=BATCH_SIZE,
                  num_target_block_errors=100, # simulate until 100 block errors occured
                  legend="Baseline",
                  soft_estimates=True,
                  max_mc_iter=30, # run 100 Monte-Carlo simulations (each with batch_size samples)
                  show_fig=False);
ber_plots.simulate(baseline16QAM2,
                  ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 41),
                  batch_size=BATCH_SIZE,
                  num_target_block_errors=100, # simulate until 100 block errors occured
                  legend="Baseline",
                  soft_estimates=True,
                  max_mc_iter=30, # run 100 Monte-Carlo simulations (each with batch_size samples)
                  show_fig=False);
ber_plots.simulate(baseline64QAM2,
                  ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 41),
                  batch_size=BATCH_SIZE,
                  num_target_block_errors=100, # simulate until 100 block errors occured
                  legend="Baseline",
                  soft_estimates=True,
                  max_mc_iter=30, # run 100 Monte-Carlo simulations (each with batch_size samples)
                  show_fig=False);

EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
    -15.0 | 4.2999e-01 | 1.0000e+00 |       68688 |      159744 |          256 |         256 |         2.5 |reached target block errors
    -14.0 | 4.0638e-01 | 1.0000e+00 |       64916 |      159744 |          256 |         256 |         0.2 |reached target block errors
    -13.0 | 3.9016e-01 | 1.0000e+00 |       62325 |      159744 |          256 |         256 |         0.2 |reached target block errors
    -12.0 | 3.6586e-01 | 1.0000e+00 |       58444 |      159744 |          256 |         256 |         0.2 |reached target block errors
    -11.0 | 3.3748e-01 | 1.0000e+00 |       53911 |      159744 |          256 |         256 |         0.3 |reached target block errors
    -10.0 | 3.0427e-01 | 1.0000e+00 |       48606 |      159744 |